In [9]:
# https://platform.openai.com/docs/api-reference


%pip install openai
%pip install tiktoken
%pip install faiss-cpu

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tiktoken (from versions: none)
ERROR: No matching distribution found for tiktoken

[notice] A new release of pip available: 22.3.1 -> 23.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import openai


# https://platform.openai.com/docs/models
openai.Model.list()


<OpenAIObject list at 0x2309db2f890> JSON: {
  "object": "list",
  "data": [
    {
      "id": "babbage",
      "object": "model",
      "created": 1649358449,
      "owned_by": "openai",
      "permission": [
        {
          "id": "modelperm-49FUp5v084tBB49tC4z8LPH5",
          "object": "model_permission",
          "created": 1669085501,
          "allow_create_engine": false,
          "allow_sampling": true,
          "allow_logprobs": true,
          "allow_search_indices": false,
          "allow_view": true,
          "allow_fine_tuning": false,
          "organization": "*",
          "group": null,
          "is_blocking": false
        }
      ],
      "root": "babbage",
      "parent": null
    },
    {
      "id": "text-davinci-003",
      "object": "model",
      "created": 1669599635,
      "owned_by": "openai-internal",
      "permission": [
        {
          "id": "modelperm-jepinXYt59ncUQrjQEIUEDyC",
          "object": "model_permission",
          "created": 1

In [4]:
openai.Model.retrieve('gpt-3.5-turbo')


<Model model id=gpt-3.5-turbo at 0x2309db2cad0> JSON: {
  "id": "gpt-3.5-turbo",
  "object": "model",
  "created": 1677610602,
  "owned_by": "openai",
  "permission": [
    {
      "id": "modelperm-ILldYg889l2lED1b99xWEfey",
      "object": "model_permission",
      "created": 1689885359,
      "allow_create_engine": false,
      "allow_sampling": true,
      "allow_logprobs": true,
      "allow_search_indices": false,
      "allow_view": true,
      "allow_fine_tuning": false,
      "organization": "*",
      "group": null,
      "is_blocking": false
    }
  ],
  "root": "gpt-3.5-turbo",
  "parent": null
}

LangChain + Faiss

- https://python.langchain.com/docs/modules/data_connection/vectorstores/
- https://python.langchain.com/docs/integrations/vectorstores/

In [18]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter


# Load the document, split it into chunks, embed each chunk and load it into the vector store.
raw_documents = TextLoader('./data/QA.txt', encoding='utf-8').load()

text_splitter = CharacterTextSplitter(
    separator='\n\n', chunk_size=200, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)

documents


[Document(page_content='怎样实名认证 $$ 点击实名认证按钮，跳转到支付宝进行实名认证\n\n怎样申请音乐技能 $$ 请详细阅读音乐技能说明书，点击已阅读，并进行技能考核，考核完成后，系统会在24H内通知考核结果', metadata={'source': './data/QA.txt'})]

In [13]:
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings


embedding = OpenAIEmbeddings()
db = FAISS.from_documents(documents, embedding=embedding)


In [14]:
db.similarity_search('实名认证', k=3)

[Document(page_content='怎样实名认证 $$ 点击实名认证按钮，跳转到支付宝进行实名认证', metadata={'source': './data/QA.txt'}),
 Document(page_content='怎样申请音乐技能 $$ 请详细阅读音乐技能说明书，点击已阅读，并进行技能考核，考核完成后，系统会在24H内通知考核结果', metadata={'source': './data/QA.txt'})]

In [15]:
embedding_vector = embedding.embed_query('音乐')
db.similarity_search_by_vector(embedding_vector)


[Document(page_content='怎样申请音乐技能 $$ 请详细阅读音乐技能说明书，点击已阅读，并进行技能考核，考核完成后，系统会在24H内通知考核结果', metadata={'source': './data/QA.txt'}),
 Document(page_content='怎样实名认证 $$ 点击实名认证按钮，跳转到支付宝进行实名认证', metadata={'source': './data/QA.txt'})]

In [ ]:
db.similarity_search_with_score('音乐', k=3)
